In [1]:
%%configure
{ "conf": {
            "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false"
          }}

In [2]:
import org.apache.spark.sql.{functions, SparkSession}
import org.apache.spark.sql.types.DataTypes
import org.apache.spark.sql.types.StructField
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.DataSourceReadOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.config.HoodieCompactionConfig
import org.apache.hudi.table.action.compact.CompactionTriggerStrategy
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.streaming.StreamingQueryListener._
import org.apache.spark.sql.{DataFrame, Row, SaveMode}
import java.time.LocalDateTime

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1647910482647_0002,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.{functions, SparkSession}
import org.apache.spark.sql.types.DataTypes
import org.apache.spark.sql.types.StructField
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.DataSourceReadOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.config.HoodieCompactionConfig
import org.apache.hudi.table.action.compact.CompactionTriggerStrategy
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.streaming.StreamingQueryListener._
import org.apache.spark.sql.{DataFrame, Row, SaveMode}
import java.time.LocalDateTime


In [5]:
val s3Bucket = "s3://emr-hudi-corrine"
val TABLE = "sales"
val tableV1DF = spark.read.format("org.apache.hudi").load(s"${s3Bucket}/${TABLE}/*/*")
tableV1DF.createOrReplaceTempView(s"${TABLE}_v1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3Bucket: String = s3://emr-hudi-corrine
TABLE: String = sales
tableV1DF: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 7 more fields]


In [7]:
val beginCommitTime = "20220322010017"
val incrementalDF = spark.read.format("org.apache.hudi").
  option(DataSourceReadOptions.QUERY_TYPE_OPT_KEY, DataSourceReadOptions.QUERY_TYPE_INCREMENTAL_OPT_VAL).
  option(DataSourceReadOptions.BEGIN_INSTANTTIME_OPT_KEY, beginCommitTime).
  load(s"${s3Bucket}/${TABLE}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

beginCommitTime: String = 20220322010017
incrementalDF: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 7 more fields]


In [8]:
incrementalDF.createOrReplaceTempView("tni_read")
val hudi_tni_read = spark.sql("select * from tni_read").limit(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hudi_tni_read: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 7 more fields]


In [9]:
hudi_tni_read.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------------------+----------------------+--------------------+-----+----------+-------+-----+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|   id|      date|   name|sales|
+-------------------+--------------------+------------------+----------------------+--------------------+-----+----------+-------+-----+
|     20220322010020|20220322010020_0_...|             14096|               Corrine|7c37a2dc-de8b-4e1...|14096|2022-03-15|Corrine|12345|
|     20220322010020|20220322010020_0_...|             14097|               Corrine|7c37a2dc-de8b-4e1...|14097|2022-03-15|Corrine|12345|
|     20220322010020|20220322010020_0_...|             14094|               Corrine|7c37a2dc-de8b-4e1...|14094|2022-03-15|Corrine|12345|
|     20220322010020|20220322010020_0_...|             14095|               Corrine|7c37a2dc-de8b-4e1...|14095|2022-03-15|Corrine|12345|
|     20220322010020|20220322010020_0_...